# use tableau to generate phreeqc input file

In [332]:
% setup environment
% clear data and number format
clear; format short e
% set path to equilibrium solver
txt=pwd; n=length(txt); rootpath=txt;
n=strfind(rootpath,"USERS"); rootpath=txt(1:n-2);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")
%delete existing phreeqc files
system("rm DATABASE.dat");
system("rm runfile.txt");
%create empty phreeqc files
system("touch DATABASE.dat");
system("touch runfile.txt");

In [333]:
Tableau=[...
{'H+'} {'e-'} {'Al+3'}       {'logK'}                     {'phase'}    {'species'} 
1       0        0               0                           0          {'H+'}
0       1        0               0                           0          {'e-'}
0       0        1               0                           0          {'Al+3'}
-1      0        0               -13.9951                    0          {'OH-'}
-2      0        1               -10.5945                    0          {'Al(OH)2+'}
-1      0        1               -4.9571                     0          {'Al(OH)+2'}
-4      0        1               -22.7                       0          {'Al(OH)4-'}
-2      0        2               -7.7902                     0          {'Al2(OH)2+4'}
-4      0        3               -13.8803                    0          {'Al3(OH)4+5'}
-32     0        13              -98.73                      0          {'Al13(OH)32+7'}
%solids
-3       0       1               -7.7560                     1          {'Al(OH)3s'}
];


In [334]:
% parse the tableau
components=Tableau(1,1:size(Tableau,2)-3); noofcomponents=length(components);
species=Tableau(2:end,size(Tableau,2)); 
phases=Tableau(2:end,size(Tableau,2)-1);
logK=Tableau(2:end,size(Tableau,2)-2);
c1=0; c2=0; %two counters
for i=1:length(species)
    tst=cell2mat(phases(i));
    if tst==0; c1=c1+1; solutionspecies(c1)=species(i); solnindex(c1)=i; end
    if tst==1; c2=c2+1; solidspecies(c2)=species(i); solidindex(c2)=i; end
end
Aall=cell2mat(Tableau(2:end,1:end-3));
Asoln=Aall; Asolid=Aall;
Asoln(solidindex,:)=[]; 
Asolid(solnindex,:)=[];
logKsoln=logK; logKsolid=logK;
logKsoln(solidindex,:)=[];
logKsolid(solnindex,:)=[];

In [335]:
MASTERSPECIES=[...
{'SOLUTION_MASTER_SPECIES\n'}
{'  \n'}
{'#element species        alk     gfw_formula     element_gfw\n'}
{'  \n'}
{'H        H+             -1.     H               1.008\n'}
{'H(0)     H2             0.0     H\n'}
{'H(1)     H+             -1.     0.0\n'}
{'E        e-             0.0     0.0             0.0\n'}
{'O        H2O            0.0     O               16.00\n'}
{'O(0)     O2             0.0     O\n'}
{'O(-2)    H2O            0.0     0.0\n'}
{'  \n'}
];
fileID=fopen('DATABASE.dat','a');
for i=1:size(MASTERSPECIES,1)
    line=cell2mat(MASTERSPECIES(i,:));
    fprintf(fileID,line)
end
% add components to the master species
for i=1:length(components)
    component=cell2mat(components(:,i));
    %extractbefore.m would work great but not in Octave yet
    for j=1:length(component)
        tst=component(j);
        if tst=="+"; break; end
        if tst=="-"; break; end
        componentnocharge(:,j)=tst;
    end
    line=[componentnocharge,'\t\t',component,'\t\t\t  0 \t 1 \t\t\t\t 1'];
    fprintf(fileID,line)
end
fprintf(fileID,' \n');
fprintf(fileID,' \n');
fclose(fileID);

In [336]:
SOLUTIONSPECIES=[...
{'SOLUTION_SPECIES\n'}
{'# use -gamma 1e10 0.0 to counteract automatic activity correction\n'}
{'  \n'}
{'H+ = H+\n'}
{'      log_k           0.000\n'}
{'      -gamma          1e10     0.0\n'}
{'  \n'}
{'e- = e-\n'}
{'        log_k           0.000\n'}
{'  \n'}
{'H2O = H2O\n'}
{'        log_k           0.000\n'}
{'  \n'}
{'H2O = OH- + H+\n'}
{'        log_k           -14.000\n'}
{'        -gamma          1e10     0.0\n'}
{'  \n'}
{'2 H2O = O2 + 4 H+ + 4 e-\n'}
{'        log_k           -86.08\n'}
{'        -gamma          1e10     0.0\n'}
{'  \n'}
{'2 H+ + 2 e- = H2\n'}
{'        log_k           -3.15\n'}
{'        -gamma          1e10     0.0\n'}
{'  \n'}
];
fileID=fopen('DATABASE.dat','a');
for i=1:size(SOLUTIONSPECIES,1)
    line=cell2mat(SOLUTIONSPECIES(i,:));
    fprintf(fileID,line)
end
% add components rxns the solutionspecies
for i=1:length(components)
    component=cell2mat(components(:,i));
    line=[component,'=',component,'\n'];
    fprintf(fileID,line)
    line=['log_k=0.000\n'];
    fprintf(fileID,line)
    line=['-gamma  1e10 0.0\n'];
    fprintf(fileID,line)
    fprintf(fileID,' \n');
end
% add species rxns
for i=noofcomponents+2:size(Asoln,1) % adjust so to not include identity at top, or OH-
    test=Asoln(i,:);
    writestr=[];
    for j=1:length(test)
        %only negative will be hydroxide in normal tableau writing because use formation constants
        if test(j)<0; Astr=num2str(abs(Asoln(i,j)));
        writestr=[Astr, "H2O "];
        end
        if test(j)>0; writestr=[writestr, " + ", cell2mat(components(j))];
        end
    end
    %for j=1:length(test)
    writestr=[writestr, " = ", cell2mat(solutionspecies(i))];
    %only negative will be hydroxide in normal tableau writing because use formation constants
    if test(1)<-1; Astr=num2str(abs(Asoln(i,1)));
        writestr=[writestr," + ",Astr, "OH- "];
    end
    fprintf(fileID,writestr);
    fprintf(fileID,' \n');
    Kstr=num2str(cell2mat(logKsoln(i)));
    line=['log_k=',Kstr,'\n'];
    fprintf(fileID,line)
    line=['-gamma  1e10 0.0\n'];
    fprintf(fileID,line)
    fprintf(fileID,' \n')
end
fclose(fileID);

In [337]:
% add solids
fileID=fopen('DATABASE.dat','a');
fprintf(fileID,'PHASES')
fprintf(fileID,' \n')
for i=1:size(Asolid,1)
    writestr=[cell2mat(solidspecies(i))];
    fprintf(fileID,writestr);
    fprintf(fileID,' \n');
    test=Asolid(i,:);
    writestr=[];
    for j=1:length(test)
        %only negative will be hydroxide in normal tableau writing because use formation constants
        if test(j)<0; Astr=num2str(abs(Asolid(i,j)));
        writestr=[Astr, "H2O "];
        end
        if test(j)>0; writestr=[writestr, " + ", cell2mat(components(j))];
        end
    end
    %for j=1:length(test)
    solidstr=cell2mat(solidspecies(i)); solidstr=solidstr(1:end-1);
    writestr=[writestr, " = ", solidstr];
    %only negative will be hydroxide in normal tableau writing because use formation constants
    if test(1)<-1; Astr=num2str(abs(Asolid(i,1)));
        writestr=[writestr," + ",Astr, "OH- "];
    end
    fprintf(fileID,writestr);
    fprintf(fileID,' \n');
    Kstr=num2str(cell2mat(logKsolid(i)));
    line=['log_k=',Kstr,'\n'];
    fprintf(fileID,line)
%    line=['-gamma  1e10 0.0\n'];
%    fprintf(fileID,line)
    fprintf(fileID,' \n')
end
fclose(fileID);